In [ ]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.8.9\n")
    
!pip install -r requirements.txt  --upgrade --user

In [1]:
import kfp
from kfp import dsl
from kfp.components import func_to_container_op, InputPath, OutputPath

  
@func_to_container_op
def get_random_int_op(minimum: int, maximum: int) -> int:
   """Generate a random number between minimum and maximum (inclusive)."""
   import random
   result = random.randint(minimum, maximum)
   print(result)
   return result

@func_to_container_op
def flip_coin_op() -> str:
    """Flip a coin and output heads or tails randomly."""
    import random
    result = random.choice(['heads', 'tails'])
    print(result)
    return result


@func_to_container_op
def print_op(message: str):
    """Print a message."""
    print(message)

@func_to_container_op
def fail_op(message):
    """Fails."""
    import sys
    print(message)    
    sys.exit(1)

def add_resource_constraints(op: dsl.ContainerOp):
    return op.set_cpu_request("100m").set_cpu_limit("100m")        
    
    
@dsl.pipeline(
    name='Conditional execution pipeline with exit handler',
    description='Shows how to use dsl.Condition() and dsl.ExitHandler().'
)
def flipcoin_exit_pipeline():
    exit_task = add_resource_constraints(print_op('Exit handler has worked!'))
    with dsl.ExitHandler(exit_task):
        flip = add_resource_constraints(flip_coin_op())
        # to disable run cache, use:
        #flip.execution_options.caching_strategy.max_cache_staleness = "P0D"
        
        with dsl.Condition(flip.output == 'heads'):
            random_num_head = add_resource_constraints(get_random_int_op(0, 9))
            with dsl.Condition(random_num_head.output > 5):
                add_resource_constraints(print_op('heads and %s > 5!' % random_num_head.output))
            with dsl.Condition(random_num_head.output <= 5):
                add_resource_constraints(print_op('heads and %s <= 5!' % random_num_head.output))

        with dsl.Condition(flip.output == 'tails'):
            random_num_tail = add_resource_constraints(get_random_int_op(10, 19))
            with dsl.Condition(random_num_tail.output > 15):
                add_resource_constraints(print_op('tails and %s > 15!' % random_num_tail.output))
            with dsl.Condition(random_num_tail.output <= 15):
                add_resource_constraints(print_op('tails and %s <= 15!' % random_num_tail.output))

        with dsl.Condition(flip.output == 'tails'):
            add_resource_constraints(fail_op(message="Failing the run to demonstrate that exit handler still gets executed."))

    

In [2]:
# generate workflow artifacts in .zip format
kfp.compiler.Compiler().compile(flipcoin_exit_pipeline, 'helloworld-1.zip')